# Scraping DeviceSpecifications.com for phone spec data with Selenium and BeautifulSoup

**Import all modules and packages, and read pickled data as a Pandas DataFrame

In [2]:
import numpy as np
import pandas as pd
import requests
import random
import pickle as pkl

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

from IPython.core.display import display, HTML
from bs4 import BeautifulSoup

In [124]:
with open('clean_scrape_data.pkl', 'rb') as picklefile:
    df = pkl.load(picklefile)

product_name = df['product_name']

In [133]:
# Create empty lists to collect data for when Selenium and BeautifulSoup are running

#memory = []
#ram = []
#camera = []
#battery = []

### Set up chromedriver, Selenium and Beautiful Soup to Web Scrape

In [4]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [180]:
driver = webdriver.Chrome(chromedriver)

driver.get('https://www.devicespecifications.com/en')

for product in product_name[401:523]:
    search_form = driver.find_element_by_id('search')
    time.sleep(random.choice(range(0,5)))
    search_form.send_keys(product)
    search_form.send_keys(Keys.RETURN)
    time.sleep(3)
    search_form.send_keys(Keys.RETURN)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    content = soup.find('div', id = 'model-brief-specifications').text
    content.strip('\n')

    ram.append(content[content.find('RAM')+4:content.find('Storage')])
    memory.append(content[content.find('Storage')+8:content.find('Storage')+25])
    battery.append(content[content.find('Battery'):content.find('OS:')][8:17])
    camera.append(content[content.find('Camera'):content.find(' fps')][-3:])

### Put all collected data (in lists) into a Pandas DataFrame

In [187]:
df1 = pd.DataFrame(columns = ['product_name', 'memory', 'ram', 'battery', 'camera'])
df1['product_name'] = product_name
df1['memory'] = memory
df1['ram'] = ram
df1['battery'] = battery
df1['camera'] = camera

df1.head(3)

,product_name,memory,ram,battery,camera
0,BlackBerry Leap 16GB Factory Unlocked GSM 4G L...,16 GBMemory card,"2 GB, 533 MHz",2800 mAh,30
1,Kyocera DuraForce E6560 16GB Unlocked GSM 4G L...,16 GBMemory card,2 GB,3100 mAh,30
2,LG G2 D800 GSM 4G LTE Unlocked Smartphone with...,16 GBMemory card,"2 GB, 667 MHz",4500 mAh,30


In [ ]:
# Save obtained DataFrame above into a pickle file

with open('part_two_scrape.pkl', 'wb') as picklefile:
    pkl.dump(df1, picklefile)